In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Generics: Introduction

<div class="alert alert-block alert-success"> 
    Source code for this notebook can be found in the package <tt>ca.queensu.cisc124.notes.generics.intro</tt>.
</div>

Java 5 introduced *generics* to the Java language. The introduction of generics was a major undertaking; great care and some significant compromises were made by the designers
of the Java language to incorporate generics without breaking the large body of existing Java code.

We have already used generic interfaces such as `List`, `Set`, `Map`, and `Comparable` and generic classes such as `ArrayList`, `HashSet`, and `HashMap`. 




This notebook 
illustrates why the designers of the Java language introduced generics to the language.

## Revisiting the `Stack` interface and `ListStack` class

The `Stack` interface and `ListStack` class were described in the [Interfaces: Stacks](../part3/interfaces_stacks.ipynb#notebook_id) notebook. The `Stack` interface shown in the following cell
describes a data structure representing a stack of strings:

In [ ]:
/**
 * The {@code Stack} interface represents a last-in-first-out (LIFO) stack of
 * strings. In addition to the usual push and pop methods, this interface
 * allows the user to get the number of strings in a stack and to query
 * if the stack is empty.
 */
public interface Stack {

    /**
     * Returns the number of elements in this stack.
     * 
     * @return the number of elements in this stack
     */
    public int size();

    /**
     * Returns {@code true} if this stack contains no elements. The default
     * implementation simply returns {@code size() == 0}.
     * 
     * @return true if this stack contains no elements
     */
    default boolean isEmpty() {
        return this.size() == 0;
    }

    /**
     * Pushes the specified element on to the top of this stack.
     * 
     * @param elem the element to be pushed on to the top of this stack
     */
    public void push(String elem);

    /**
     * Removes the element on the top of this stack and returns the element.
     * 
     * @return the top element of this stack
     * @throws RuntimeException if the stack is empty
     */
    public String pop();
}

`ListStack` is a stack implementation that uses an `ArrayList` to store the elements of the stack:

In [ ]:
import java.util.ArrayList;

public class ListStack implements Stack {

    private ArrayList<String> stack;
    
    public ListStack() { 
        this.stack = new ArrayList<>();
    }

    @Override 
    public int size() {
        return this.stack.size();
    }

    @Override
    public void push(String elem) {
        this.stack.add(elem);
    }

    @Override
    public String pop() {
        if (this.isEmpty()) {
            throw new RuntimeException("popped an empty stack");
        }
        String elem = this.stack.remove(this.size() - 1);
        return elem;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}

**Exercise 1:** Examine the constructor and the methods of the `ListStack` class. Do any of constructors or methods directly call a `String` method or constructor? (answer follows)

While the `ListStack` class represents a stack of strings, none of the constructors or methods directly call a `String` method or constructor. This means that we should be able to
implement a stack holding elements of a different type with only very minor modifications to the current `ListStack` implementation.

**Exercise 2:** Implement a class representing a stack of `Integer` elements. (answer follows)

In [ ]:
// Exericise 2

import java.util.ArrayList;

public class ListIntStack {
    
}

Implementing a stack-of-integers class requires seven changes to the existing `ListStack` class:

1. change the name of the class (so that the different stack classes have different names)
2. remove `implements Stack` because the `Stack` interface describes a stack of strings
3. change the field type to `ArrayList<Integer>`
4. change the constructor name to match the class name
5. change the parameter type of `push` to `Integer`
6. change the return type of `pop` to `Integer`
7. change the type of the local variable `s` in `pop`

A fully implemented stack-of-integers class with the modifications marked by comments is shown below:

In [ ]:
import java.util.ArrayList;

public class ListIntStack {                                   // 1, 2

    private ArrayList<Integer> stack;                         // 3
    
    public ListIntStack() {                                   // 4
        this.stack = new ArrayList<>();
    }

    public int size() {
        return this.stack.size();
    }
    
    public boolean isEmpty() {
        return this.size() == 0;
    }

    public void push(Integer elem) {                          // 5 change method parameter type from String to Integer
        this.stack.add(elem);
    }

    public Integer pop() {                                    // 6 change return type from String to Integer
        if (this.isEmpty()) {
            throw new RuntimeException("popped an empty stack");
        }
        Integer elem = this.stack.remove(this.size() - 1);    // 7 change local variable type from String to Integer
        return elem;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("Stack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}

For the moment, ignore the fact that `ListIntStack` no longer implements the `Stack` interface (although this is a significant drawback because we can no longer pass a `ListIntStack`
object to a method that takes a `Stack` reference).

Focusing on changes 5, 6, and 7, notice that the only modifications required to change the implementation from a stack-of-strings to a stack-of-integers are modifying some type names.
It would be easy to create different
stack classes for different element types simply by copying `ListStack` and making the modifications described above, but we **really** want to avoid doing so because we would end up duplicating
almost all of the code everytime we wanted a stack of some other element type.

Before generics were introduced to the Java langauge, code duplication was avoided by implementing a stack of `Object`s:

In [ ]:
import java.util.ArrayList;

public class ListObjectStack {

    private ArrayList<Object> stack;                         // list of Object
    
    public ListObjectStack() {
        this.stack = new ArrayList<>();
    }

    public int size() {
        return this.stack.size();
    }
    
    public boolean isEmpty() {
        return this.size() == 0;
    }

    public void push(Object elem) {                          // push an object onto the stack
        this.stack.add(elem);
    }

    public Object pop() {                                    // pop an object from the stack
        if (this.isEmpty()) {                  // added to satisfy the updated contract of pop
            throw new RuntimeException("popped an empty stack");
        }
        Object elem = this.stack.remove(this.size() - 1);    // remove an object from the end of the list
        return elem;
    }

    @Override
    public String toString() {
        StringBuilder b = new StringBuilder("ObjectStack:");
        if (this.size() != 0) {
            for (int i = this.size() - 1; i >= 0; i--) {
                b.append('\n');
                b.append(this.stack.get(i));
            }
        }
        return b.toString();
    }
}

Such a solution is inconvenient for the user of the class because the user must cast the reference returned by `pop` if they want to use the reference to invoke a method:

In [ ]:
ListObjectStack stringStack = new ListObjectStack();
stringStack.push("hello");
stringStack.push("konnichiwa");
stringStack.push("hallo");

String s = (String) stringStack.pop();      // cast needed because pop returns an Object reference
System.out.println(s.toUpperCase());

A more significant problem with the stack of `Object` approach is the lack of type safety: *Any* reference can be pushed onto such a stack:

In [ ]:
import java.util.Date;

ListObjectStack stringStack = new ListObjectStack();
stringStack.push("hello");
stringStack.push("konnichiwa");
stringStack.push("hallo");
stringStack.push(1);                           // works, 1 is autoboxed to an Integer object
stringStack.push(new ListObjectStack());       // works
stringStack.push(new Date());                  // works
System.out.println(stringStack);

In the example above, `stringStack` is a stack that contains references to three `String` objects, an `Integer` object, a `Stack` object, and a `Date` object. There is no way
for the programmer to prevent non-string references from being pushed onto the stack.

The lack of type safety makes it impossible to write type-safe methods. The following cell shows a method that sums the elements of a stack assuming that the elements
in the stack are all castable to the type `Integer`:

In [ ]:
public class Sum {
    
    /**
     * Returns the sum of the elements in a stack of int (or byte, char, short) or
     * zero if the stack is empty. Removes all elements from the specified stack.
     *
     * <p>
     * The returned sum may overflow the range of int.
     *
     * @param intStack a stack of Integer
     * @return the sum of the elements in the stack
     * @throws ClassCastException if the stack contains non-Integer references
     */
    public static int sumAndClear(ListObjectStack intStack) {
        int sum = 0;
        while (intStack.size() > 0) {
            Object elem = intStack.pop();
            sum += (Integer) elem;      // cast needed because pop returns an Object reference
        }
        return sum;
    }
}

The method works fine if the stack contains only `Integer` references:

In [ ]:
ListObjectStack s = new ListObjectStack();
s.push(1);
s.push(2);
s.push(3);
System.out.println(Sum.sumAndClear(s));

However, the method fails at *runtime* if it is passed a stack containing a non-`Integer` reference because the cast fails (run the next cell to see the error):

In [ ]:
ListObjectStack s = new ListObjectStack();
s.push(1);
s.push("hello");
s.push(2);
System.out.println(Sum.sumAndClear(s));

The Jupyter notebook obscures the fact that the previous example compiles cleanly (without even a warning). The programmer has no indication that there is an error in their code until the program is run.

The next notebook illustrates how to use Java's generics facility to provide a typesafe way to create a generic class that avoids code duplication.